#### Mohammad Mahdi
#### Abdolhosseini
#### 810 198 434
#### CA2_Part2_Game

In [1]:
from select import select
from symbol import dotted_as_name
import turtle
import math
import random
from time import sleep
from sys import argv
import time

In [2]:
class Sim:
    # Set true for graphical interface
    GUI = False
    screen = None
    selection = []
    turn = ''
    dots = []
    red = []
    blue = []
    available_moves = []
    minimax_depth = 0
    prune = False

    def __init__(self, minimax_depth, prune, gui):
        self.GUI = gui
        self.prune = prune
        self.minimax_depth = minimax_depth
        if self.GUI:
            self.setup_screen()

    def setup_screen(self):
        self.screen = turtle.Screen()
        self.screen.setup(800, 800)
        self.screen.title("Game of SIM")
        self.screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self.screen.tracer(0, 0)
        turtle.hideturtle()

    def draw_dot(self, x, y, color):
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def gen_dots(self):
        r = []
        for angle in range(0, 360, 60):
            r.append((math.cos(math.radians(angle)), math.sin(math.radians(angle))))
        return r

    def initialize(self):
        self.selection = []
        self.available_moves = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    self.available_moves.append((i, j))
        if random.randint(0, 2) == 1:
            self.turn = 'red'
        else:
            self.turn = 'blue'
        self.dots = self.gen_dots()
        self.red = []
        self.blue = []
        if self.GUI: turtle.clear()
        self.draw()

    def draw_line(self, p1, p2, color):
        turtle.up()
        turtle.pensize(3)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def draw_board(self):
        for i in range(len(self.dots)):
            if i in self.selection:
                self.draw_dot(self.dots[i][0], self.dots[i][1], self.turn)
            else:
                self.draw_dot(self.dots[i][0], self.dots[i][1], 'dark gray')

    def draw(self):
        if not self.GUI: return 0
        self.draw_board()
        for i in range(len(self.red)):
            self.draw_line((math.cos(math.radians(self.red[i][0] * 60)), math.sin(math.radians(self.red[i][0] * 60))),
                           (math.cos(math.radians(self.red[i][1] * 60)), math.sin(math.radians(self.red[i][1] * 60))),
                           'red')
        for i in range(len(self.blue)):
            self.draw_line((math.cos(math.radians(self.blue[i][0] * 60)), math.sin(math.radians(self.blue[i][0] * 60))),
                           (math.cos(math.radians(self.blue[i][1] * 60)), math.sin(math.radians(self.blue[i][1] * 60))),
                           'blue')
        self.screen.update()
        sleep(0.1)
    
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#returns the actual minimax value of the position
    #def _evaluate(self):
        #TODO
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    def minimax(self, depth, player_turn):
        bestVal = -float("inf")
        bestMove = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    if (i,j) in self.available_moves:
                        self.red.append((i,j))
                        self.available_moves.remove((i,j))
                        val = self.minimaxValue(1, "red", -float("inf"), float("inf"))
                        if val > bestVal:
                            bestVal = val
                            bestMove = (i,j) 
                        self.available_moves.append((i,j))
                        self.red.remove((i,j))                    
        return bestMove
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    def maxValue(self, depth, alpha, beta):
        v = -float("inf")
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    if (i,j) in self.available_moves:
                        self.red.append((i,j))
                        self.available_moves.remove((i,j))
                        #MIN:
                        v = max(v, self.minimaxValue(depth+1, "blue", alpha, beta))
                        self.available_moves.append((i,j))
                        self.red.remove((i,j))
                        alpha = max(alpha, v)
                        #Alpha-Beta Pruning:
                        if (v >= beta) and (self.prune == True):
                            return v
        return v
    
    def minValue(self, depth, alpha, beta):    
        v = float("inf")
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    if (i,j) in self.available_moves:
                        self.blue.append((i,j))
                        self.available_moves.remove((i,j))
                        #MAX:
                        v = min(v, self.minimaxValue(depth+1, "red", alpha, beta))
                        self.available_moves.append((i,j))
                        self.blue.remove((i,j))
                        beta = min(beta , v)
                        #Alpha-Beta Pruning:
                        if (v <= alpha) and (self.prune == True):
                            return v
        return v
     
    def minimaxValue(self, depth, player_turn, alpha, beta):

        v = depth
        gameResult = self.gameover(self.red,self.blue)
        if gameResult == "red": v += 10
        elif gameResult == "blue": v += -10

        if gameResult != 0:
            return v

        if depth >= self.minimax_depth:
            return v
        
        if player_turn == "red":
            return self.maxValue(depth, alpha, beta)
        
        elif player_turn == "blue":
            return self.minValue(depth, alpha, beta)
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

    def enemy(self):
        return random.choice(self.available_moves)

    def _swap_turn(self, turn):
        if turn == "red": return "blue"
        else: return "red"
    
    def play(self):
        self.initialize()
        while True:
            if self.turn == 'red':
                selection = self.minimax(depth=self.minimax_depth, player_turn=self.turn)#[0]
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            else:
                selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            if selection in self.red or selection in self.blue:
                raise Exception("Duplicate Move!!!")
            if self.turn == 'red':
                self.red.append(selection)
            else:
                self.blue.append(selection)

            self.available_moves.remove(selection)
            self.turn = self._swap_turn(self.turn)
            selection = []
            self.draw()
            r = self.gameover(self.red, self.blue)
            if r != 0:
                return r

    def gameover(self, r, b):
        if len(r) < 3:
            return 0
        r.sort()
        for i in range(len(r) - 2):
            for j in range(i + 1, len(r) - 1):
                for k in range(j + 1, len(r)):
                    if r[i][0] == r[j][0] and r[i][1] == r[k][0] and r[j][1] == r[k][1]:
                        return 'blue'
        if len(b) < 3: return 0
        b.sort()
        for i in range(len(b) - 2):
            for j in range(i + 1, len(b) - 1):
                for k in range(j + 1, len(b)):
                    if b[i][0] == b[j][0] and b[i][1] == b[k][0] and b[j][1] == b[k][1]:
                        return 'red'
        return 0  

In [3]:
prune = (False, True)
depth = (1, 3, 5, 7)
for p in prune:
    for d in depth:
        if (p==False and d>=7) == False : #if prune=false and d=7, run time will be significantly high.
            t1 = time.time()
            game = Sim(minimax_depth=d, prune=p, gui=0)
            results = {"red": 0, "blue": 0}
            for i in range(100):
                #print(i)
                results[game.play()] += 1
            t2 = time.time()
            print("Results for minimax_depth=" + str(d) + " and prune=" + str(p) + " :\t" + str(results))
            print("Elapsed Time:\t" + str(t2-t1) + "\n")       

Results for minimax_depth=1 and prune=False :	{'red': 84, 'blue': 16}
Elapsed Time:	0.02592945098876953

Results for minimax_depth=3 and prune=False :	{'red': 91, 'blue': 9}
Elapsed Time:	1.6566646099090576

Results for minimax_depth=5 and prune=False :	{'red': 92, 'blue': 8}
Elapsed Time:	189.65148878097534

Results for minimax_depth=1 and prune=True :	{'red': 87, 'blue': 13}
Elapsed Time:	0.02289581298828125

Results for minimax_depth=3 and prune=True :	{'red': 90, 'blue': 10}
Elapsed Time:	0.5425500869750977

Results for minimax_depth=5 and prune=True :	{'red': 91, 'blue': 9}
Elapsed Time:	7.578460216522217

Results for minimax_depth=7 and prune=True :	{'red': 93, 'blue': 7}
Elapsed Time:	77.2142186164856



سوالات:

سوال1: یک هیوریستیک خوب باید بتواند تحت شرایط مختلفی که در بازی به وجود میاید، بهترین مسیر را برای پیروزی ارائه دهد. در این الگوریتم، به ازای پیروزی مقدار مثبت 10 و شکست مقدار منفی 10 را درنظر میگیریم. همچنین در هر مرحله که جلو میرویم، متناسب با عمق طی شده، مقداری را به امتیازات 
اضافه میکنیم. مثلا برای پیروزی در عمق 5 امتیاز 15 و برای شکست در عمق 7 امتیاز -3 ثبت میشود

سوال2: مطابق نتایج، با افزایش عمق الگوریتم، شانش پیروزی افزایش میابد، زیرا گره های بیشتری دیده شده و بهترین مسیر از میان آنها انتخاب شده. همچنین همانطور که از نتایج پیداست به واسطه مشاهده گره های بیشتر، زمان اجرا نیز افزایش میابد

سوال3: زمانی که از روش هرس کردن استفاده میکنیم، زمان اجرا بسیار بهبود میابد. در این الگوریتم، ترتیب اضافه شدن فرزندان به هر گره، مشابه حالت بدون هرس میباشد. در واقع به ترتیب اضافه شدن از آنجا که شرایط برنده شدن بصورت رندم است، آنچنان اهمیت نخواهد داشت و برای ساده تر شدن کار تغییری داده نشده. البته  بهتر کار با گرهی که بتای کمتر و آلفای بیشتری دارد شروع شود   